In [2]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from datasets import load_dataset
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics import f1_score
from torch import nn

from torch.utils.data import DataLoader
from tqdm import tqdm

# Deep Average Network для определения сентимента 

В этой домашке мы будет классифицировать твиты на 3 тональности.  
Вы будете использовать предобученные эмбеддинги слов, так что для начала обязательно нужно посмотреть [туториал по их использованию](https://github.com/BobaZooba/DeepNLP/blob/master/Tutorials/Word%20vectors%20%26%20Data%20Loading.ipynb).

Наши классы:  

Индекс | Sentiment  
-- | --  
0 | negative  
1 | neutral  
2 | positive  

Вам предстоит реализовать такую модель:
![Архитектура модели DAN](https://www.researchgate.net/profile/Shervin-Minaee/publication/340523298/figure/fig1/AS:878252264550411@1586403065555/The-architecture-of-the-Deep-Average-Network-DAN-10.ppm)

Что она из себя представляет:
- Мы подаем в нее индексы слов
- Переводим индексы слов в эмбеддинги
- Усредняем эмбеддинги
- Пропускаем усредненные эмбеддинги через `Multilayer Perceptron`

В этой домашке вам предстоит:
- Перевести тексты в матрицы с индексами токенов
- Реализовать модель
- Обучить ее
- Понять хорошо ли вы это сделали

Это очень важная модель, потому что она очень простая и показывает достаточно высокие метрики. В дальнейшем на работе советую использовать такую модель как бейзлайн. И в качестве эмбеддингов слов взять эмбеддинги от берта/роберты/тд.

## 🤗 Datasets
В этом туториале мы будем использовать подготовленные данные из библиотеки [datasets](https://github.com/huggingface/datasets). Мы вряд ли еще будем пользоваться этой библиотекой, так как нам будет важно самим подготавливать данные. Во-первых, для простоты, во-вторых, здесь есть достаточно неплохие практики. [Здесь](https://huggingface.co/datasets) вы сможете найти достаточно большое количество различных датасетов. Возможно, когда-нибудь они вам пригодятся.

## Загрузите эмбеддинги слов
Реализуйте функцию по загрузке эмбеддингов из файла. Она должна отдавать словарь слов и `np.array`
Формат словаря:
```python
{
    'aabra': 0,
    ...,
    'mom': 6546,
    ...
    'xyz': 100355
}
```
Формат матрицы эмбеддингов:
```python
array([[0.44442278, 0.28644582, 0.04357426, ..., 0.9425766 , 0.02024289,
        0.88456545],
       [0.77599317, 0.35188237, 0.54801261, ..., 0.91134102, 0.88599103,
        0.88068835],
       [0.68071886, 0.29352313, 0.95952505, ..., 0.19127958, 0.97723054,
        0.36294011],
       ...,
       [0.03589378, 0.85429694, 0.33437761, ..., 0.39784873, 0.80368014,
        0.76368042],
       [0.01498725, 0.78155695, 0.80372969, ..., 0.82051826, 0.42314861,
        0.18655465],
       [0.69263802, 0.82090775, 0.27150426, ..., 0.86582747, 0.40896573,
        0.33423976]])
```

Количество строк в матрице эмбеддингов должно совпадать с размером словаря, то есть для каждого токена должен быть свой эмбеддинг. По параметру `num_tokens` должно брать не более указано в этом параметре количество токенов в словарь и матрицу эмбеддингов.

In [3]:
# Раскомментируйте и скачайте эмбеддинги
# !wget  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
# !gzip -d cc.ru.300.vec.gz

In [4]:
token2index: Dict[str, int] = {}
embeddings_matrix: np.array = []

with open("small.vec", "r") as f:
    lines = f.readlines()
    for idx, line in enumerate(lines):
        if idx == 0: continue
        line = line.split(" ")
        word, embedd = line[0],  line[1:]
        
        token2index[word] = idx-1
        embeddings_matrix.append(embedd)

        if idx == 30: break


In [5]:
np.array(embeddings_matrix, dtype=float)

array([[-0.0326, -0.1499,  0.0232, ..., -0.0237, -0.0261, -0.1832],
       [-0.0555, -0.0175,  0.0936, ...,  0.0201,  0.0535, -0.3563],
       [-0.0312, -0.0627,  0.0326, ...,  0.0398, -0.2425, -0.2647],
       ...,
       [-0.0044, -0.1126, -0.026 , ...,  0.1368, -0.2307, -0.3249],
       [-0.063 ,  0.0324, -0.0932, ...,  0.1149,  0.0648,  0.0566],
       [ 0.1825, -0.0233, -0.0524, ..., -0.0023, -0.1985,  0.0598]])

In [6]:
def load_embeddings(path, num_tokens=100_000):

    # Необязательно задавать здесь
    # Это рекомендация к типу
    token2index: Dict[str, int] = {}
    embeddings_matrix: np.array = []
    pad_token = 'PAD'
    unk_token = 'UNK'

    token2index[pad_token] = 0
    embeddings_matrix.append(np.zeros(300))

    token2index[unk_token] = 1
    embeddings_matrix.append(np.zeros(300))
            
    with open(path, "r") as f:
        lines = f.readlines()
    
    for idx, line in enumerate(lines):
            if idx == 0: continue
            line = line.split(" ")
            word, embedd = line[0],  line[1:]
            
            token2index[word] = idx-1
            embeddings_matrix.append(embedd)
   
    embeddings_matrix = np.array(embeddings_matrix, dtype=float)
    assert(len(token2index) == embeddings_matrix.shape[0])
    
    return token2index, embeddings_matrix



In [7]:
token2index, embeddings_matrix = load_embeddings("small.vec")

In [8]:
token2index

{'PAD': 0,
 'UNK': 1,
 ',': 0,
 '.': 1,
 'и': 2,
 'в': 3,
 '</s>': 4,
 ':': 5,
 ')': 6,
 '(': 7,
 'на': 8,
 '"': 9,
 'с': 10,
 'не': 11,
 '»': 12,
 'для': 13,
 '-': 14,
 '«': 15,
 '/': 16,
 'по': 17,
 '—': 18,
 'что': 19,
 'В': 20,
 '!': 21,
 'из': 22,
 'от': 23,
 'к': 24,
 'как': 25,
 '?': 26,
 'а': 27,
 '–': 28,
 'за': 29,
 ';': 30,
 'о': 31,
 'или': 32,
 'это': 33,
 '�': 34,
 '1': 35,
 '...': 36,
 '_': 37,
 "'": 38,
 'его': 39,
 'у': 40,
 'до': 41,
 '|': 42,
 '2': 43,
 'но': 44,
 'все': 45,
 'года': 46,
 '+': 47,
 'я': 48,
 'то': 49,
 '%': 50,
 'при': 51,
 'он': 52,
 'так': 53,
 'же': 54,
 'только': 55,
 '3': 56,
 'их': 57,
 'А': 58,
 'можно': 59,
 '#': 60,
 'был': 61,
 'И': 62,
 'время': 63,
 'также': 64,
 'году': 65,
 '10': 66,
 'было': 67,
 'будет': 68,
 'может': 69,
 'вы': 70,
 '0': 71,
 'уже': 72,
 '[': 73,
 ']': 74,
 '4': 75,
 '>': 76,
 'чтобы': 77,
 'есть': 78,
 'На': 79,
 'если': 80,
 '5': 81,
 '…': 82,
 'г': 83,
 'которые': 84,
 'С': 85,
 'без': 86,
 'со': 87,
 'очень': 88,

## Загружаем данные из библиотеки
Мы сразу получим `torch.utils.data.Dataset`, который сможем передать в `torch.utils.data.DataLoader`

In [9]:
dataset_path = "tweet_eval"
dataset_name = "sentiment"

train_dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")
valid_dataset = load_dataset(path=dataset_path, name=dataset_name, split="validation")
test_dataset = load_dataset(path=dataset_path, name=dataset_name, split="test")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset tweet_eval (/home/jovyan/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
Found cached dataset tweet_eval (/home/jovyan/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
Found cached dataset tweet_eval (/home/jovyan/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


## `torch.utils.data.DataLoader`

In [10]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

## Посмотрим что отдает нам `Loader`
Это батч формата:
```python
batch = {
    "text": [
        "text1",
        "text2",
        ...,
        "textn"
    ],
    "label": tensor([
        1,
        1,
        ...,
        0
    ])
}
```
То есть у нас есть словарь с двумя ключами `text` и `label`, где хранится n примеров. То есть для 5-го примера в батче текст будет храниться в `batch["text"][5]`, а индекс класса будет храниться в `batch["label"][5]`.

In [11]:
for batch in train_loader:
    break

batch

{'text': ['ICC announces teams for WCL Division 3: Dubai: The International Cricket Council (ICC) Thursday announced the ...',
  "Wasn't there an International Women's day on the 8th of March too?"],
 'label': tensor([1, 1])}

## Collate
Сейчас перед нами стоит проблема: мы получаем тексты в виде строк, а нам нужны тензоры (матрицы) с индексами токенов, к тому же нам нужно западить последовательности токенов, чтобы все сложить в торчовую матрицу. Мы можем сделать это двумя способами:
- Достать из `train/valid/test_dataset` данные и написать свой `Dataset`, где внутри будет токенизировать текст, токены будут переводиться в индексы и затем последовательность будет падиться до нужной длины
- Сделать функцию, которая бы дополнительно обрабатывали наши батчи. Она вставляется в `DataLoader(collate_fn=<ВАША_ФУНКЦИЯ>)`

## Если вы хотите сделать свой `Dataset`
То вы можете достать данные таким образом.

In [12]:
len(train_dataset["text"]), len(train_dataset["label"])

(45615, 45615)

In [13]:
train_dataset["text"][:2]

['"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"',
 '"Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ"']

In [14]:
train_dataset["label"][:2]

[2, 1]

## Если вы хотите сделать `collate_fn`

### Давайте посмотрим что вообще происходит внутри этого метода
Для этого сделаем функцию `empty_collate`, которая принимает на вход батч и отдает его, ничего с ним не делая

In [15]:
def empty_collate(batch):
    return batch

In [16]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=empty_collate)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=empty_collate)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=empty_collate)

In [17]:
for batch in train_loader:
    break

batch

[{'text': 'Face like miss America... Body like a stripper... Fuck me like a pornstar... N handle business like the 1st lady... Yea i need that...$$$',
  'label': 2},
 {'text': '@user @user By the IRA? On bloody sunday 13 innocent demonstrators where gunned down in cold blood.',
  'label': 0}]

## Формат батча
```python
batch = [
    {
        "text": "text1",
        "label": 0
    }, 
    {
        "text": "text2",
        "label": 1
    },
    ...,
    {
        "text": "textn",
        "label": 1
    }
]
```
То есть теперь у нас есть список, где каждый элемент — это словарь со значениями `text` и `label`.  

Вы можете сделать функцию или класс с методом `collate`. Этот способ решения домашки предодчтительней, так как использовать `collate` очень хорошая практика.

Что я предлагаю:
- Сделайте класс `Tokenizer`

In [18]:
class Tokenizer:
    
    def __init__(self, base_tokenizer, token2index, pad_token, unk_token, max_length):
        
        self._base_tokenizer = ToktokTokenizer()  # например ToktokTokenizer()
        
        self.token2index = token2index  # словарь из load_embeddings()
        
        self.pad_token = pad_token
        self.pad_index = self.token2index[self.pad_token]
        
        self.unk_token = unk_token
        self.unk_index = self.token2index[self.unk_token]
        
        self.max_length = max_length

    def tokenize(self, text):
        """
        В этом методе нужно разделить строку текста на токены
        """
        return self._base_tokenizer.tokenize(text)
    
    def indexing(self, tokenized_text):
        """
        В этом методе нужно перевести список токенов в список с индексами этих токенов
        """
        token_indices = list()
        for i, v in enumerate(tokenized_text):
            if v in self.token2index:
                token_indices.append(self.token2index[v])
            else:
                token_indices.append(self.unk_index)
        return token_indices
        
    def padding(self, tokens_indices):
        """
        В этом методе нужно сделать длину tokens_indices равной self.max_length
        Опционально убрать повторяющиеся unk'и
        """
        if len(tokens_indices) > self.max_length:
            tokens_indices = tokens_indices[:self.max_length]
       
        if len(tokens_indices) < self.max_length:
            tokens_indices = tokens_indices + np.zeros(self.max_length - len(tokens_indices), dtype=int).tolist()

        return tokens_indices




        # tokens_indices = tokens_indices[:self.max_length]
        # return tokens_indices
    
    def __call__(self, text):
        """
        В этом методе нужно перевести строку с текстом в вектор с индексами слов нужно размера (self.max_length)
        """
        # if list(token2index.keys()) in text.split(" "):
        #     text_indicies = 
        # text = np.array(token2index)
        # return 
        tokenized_text = self.tokenize(text)
        tokens_indices = self.indexing(tokenized_text)
        tokens_indices = self.padding(tokens_indices)
        return tokens_indices
       

        
    def collate(self, batch):
        
        tokenized_texts = list()
        labels = list()
        
        for sample in batch:
            tokenized_texts.append(self.__call__(sample["text"]))
            labels.append(sample["label"])


        tokenized_texts = torch.tensor(tokenized_texts) # перевод в torch.Tensor
        labels = torch.tensor(labels) # перевод в torch.Tensor
        
        return tokenized_texts, labels

In [19]:
embeddings_matrix[3]

array([-5.5500e-02, -1.7500e-02,  9.3600e-02,  2.4500e-02, -7.0800e-02,
        4.4900e-02,  2.8900e-02,  9.9900e-02, -6.1000e-03, -1.2660e-01,
        6.4600e-02, -1.6190e-01,  1.3480e-01, -3.5000e-02,  2.9000e-03,
       -8.3100e-02, -1.9700e-02,  9.5500e-02,  2.6500e-02, -7.1400e-02,
        4.5000e-02, -1.9740e-01,  1.8700e-02, -2.5000e-03, -2.8500e-02,
       -5.5700e-02, -6.0000e-04,  9.2600e-02,  6.5000e-03,  2.6000e-03,
       -1.5610e-01, -5.2200e-02, -5.2000e-03,  1.4440e-01, -1.5500e-02,
        7.7900e-02, -9.4400e-02, -1.0973e+00,  7.1200e-02,  2.4000e-03,
        1.9500e-02, -1.3380e-01,  3.1000e-03,  2.2200e-02, -1.2160e-01,
       -2.4000e-03, -7.0900e-02, -2.4600e-02, -9.2600e-02, -1.4630e-01,
       -2.0200e-02, -5.1600e-02, -4.9600e-02,  1.3520e-01,  7.9600e-02,
       -7.4700e-02,  1.7600e-02,  1.1940e-01, -1.6950e-01,  4.8000e-02,
       -5.9600e-01, -8.5400e-02,  1.0160e-01, -1.3600e-02, -2.9600e-02,
       -2.1300e-02,  4.4000e-02,  8.9500e-02, -4.9000e-03, -5.77

In [20]:
token2index

{'PAD': 0,
 'UNK': 1,
 ',': 0,
 '.': 1,
 'и': 2,
 'в': 3,
 '</s>': 4,
 ':': 5,
 ')': 6,
 '(': 7,
 'на': 8,
 '"': 9,
 'с': 10,
 'не': 11,
 '»': 12,
 'для': 13,
 '-': 14,
 '«': 15,
 '/': 16,
 'по': 17,
 '—': 18,
 'что': 19,
 'В': 20,
 '!': 21,
 'из': 22,
 'от': 23,
 'к': 24,
 'как': 25,
 '?': 26,
 'а': 27,
 '–': 28,
 'за': 29,
 ';': 30,
 'о': 31,
 'или': 32,
 'это': 33,
 '�': 34,
 '1': 35,
 '...': 36,
 '_': 37,
 "'": 38,
 'его': 39,
 'у': 40,
 'до': 41,
 '|': 42,
 '2': 43,
 'но': 44,
 'все': 45,
 'года': 46,
 '+': 47,
 'я': 48,
 'то': 49,
 '%': 50,
 'при': 51,
 'он': 52,
 'так': 53,
 'же': 54,
 'только': 55,
 '3': 56,
 'их': 57,
 'А': 58,
 'можно': 59,
 '#': 60,
 'был': 61,
 'И': 62,
 'время': 63,
 'также': 64,
 'году': 65,
 '10': 66,
 'было': 67,
 'будет': 68,
 'может': 69,
 'вы': 70,
 '0': 71,
 'уже': 72,
 '[': 73,
 ']': 74,
 '4': 75,
 '>': 76,
 'чтобы': 77,
 'есть': 78,
 'На': 79,
 'если': 80,
 '5': 81,
 '…': 82,
 'г': 83,
 'которые': 84,
 'С': 85,
 'без': 86,
 'со': 87,
 'очень': 88,

In [21]:
batch

[{'text': 'Face like miss America... Body like a stripper... Fuck me like a pornstar... N handle business like the 1st lady... Yea i need that...$$$',
  'label': 2},
 {'text': '@user @user By the IRA? On bloody sunday 13 innocent demonstrators where gunned down in cold blood.',
  'label': 0}]

In [22]:
batch[0]['text']

'Face like miss America... Body like a stripper... Fuck me like a pornstar... N handle business like the 1st lady... Yea i need that...$$$'

## Перед реализацией выбранного метода
Советую, чтобы в итоге `Loader` отдавал кортеж с двумя тензорами:
- `torch.Tensor` с индексами токенов, размерность `(batch_size, sequence_length)`
- `torch.Tensor` с индексами таргетов, размерность `(batch_size)`

То есть, чтобы было так:
```python
for x, y in train_loader:
    ...

>> x
>> tensor([[   37,  3889,   470,  ...,     0,     0,     0],
           [ 1509,   581,   144,  ...,     0,     0,     0],
           [ 1804,   893,  2457,  ...,     0,     0,     0],
           ...,
           [  170, 39526,  2102,  ...,     0,     0,     0],
           [ 1217,   172, 28440,  ...,     0,     0,     0],
           [   37,    56,   603,  ...,     0,     0,     0]])

>> y
>> tensor([1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 0, 1,
           0, 2, 1, 1, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 0,
           1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1, 0, 0, 2, 2,
           2, 1, 2, 0, 2, 2, 0, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 0, 2, 2,
           2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 1, 2, 1,
           2, 1, 1, 2, 2, 1, 1, 2])

>> x.shape
>> torch.Size([128, 64])

>> y.shape
>> torch.Size([128])
```
При условии, что батч сайз равен 128, а максимальная длина последовательности равна 64.

## Помните

## <Место для реализации>

In [44]:
tokenizer = Tokenizer(ToktokTokenizer, token2index, 'PAD', 'UNK', 100)

In [45]:
tokenizer.collate(batch)

(tensor([[ 1,  1,  1,  1, 36,  1,  1,  1,  1, 36,  1,  1,  1,  1,  1, 36,  1,  1,
           1,  1,  1,  1,  1, 36,  1,  1,  1,  1, 36,  1,  1,  1,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 1,  1,  1,  1,  1, 26,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
           1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]),
 tensor([2, 0]))

In [46]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=tokenizer.collate)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True, collate_fn=tokenizer.collate)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, collate_fn=tokenizer.collate)

In [47]:
for x, y in train_loader:
    break

In [48]:
assert(isinstance(x, torch.Tensor))
assert(len(x.size()) == 2)

assert(isinstance(y, torch.Tensor))
assert(len(y.size()) == 1)

# Реализация DAN

На вход модели будут подавать индексы слов

Шаги:
- Переводим индексы слов в эмбеддинги
- Усредняем эмбеддинги
- Пропускаем усредненные эмбеддинги через `Multilayer Perceptron`
    - Нужно реализовать самому

### Что нужно сделать в домашке с точки зрения архитектуры сети:
- Реализовать skip-connection (residual connection) в линейном слое
- Написать свой слой, в котором будут (порядок слоев ниже напутан, так что сами подумайте в каком порядке стоит расположить эти слои) :
  - `Dropout`
  - `BatchNorm` / `LayerNorm`
  - `Residual`, если вы не меняете размерность векторов
  - Функция активации
  - Линейный слой

### Опциональные задания:
- Использовать токенизатор и слой эмбеддингов от предобученного трансформера из библиотеки `transformers`
- Сделать усреднение эмбеддингов с учетом падов
  - Мы используем пады, чтобы сделать единую длину последовательностей в батче
    - То есть у нас максимальная длина в батче, например, 16 токенов, поэтому ко всем последовательностям, у которых длина ниже мы добавляем `16 - len(sequence)` падов
  - То есть получается так, что усредненный вектор предложения зависит от максимальный длины в батче, потому что
    - Среднее вектора `[1, 2, 3]` будет `2`. Среднее вектора `[1, 2, 3, 0, 0]` будет `1.2`
    - Получается, что усредняя с падами мы получаем "неправильный" вектор
  - То есть наши предсказания будут зависеть от того сколько падов у нас есть в предложении
  - Когда мы будем использовать нашу сетку в реальном процессе, скорее всего, мы будем подавать в нее по одному примеру, где падов не будет
    - То есть получается мы будем использовать нашу модель не в той же среде, как и обучали
      - Потому что наши входы меняются, мы не используем пады, результат усреднения другой
    - Это называется `distribution shift`, то есть когда мы учимся на одних данных, а используем на других
      - Это не всегда плохо, потому что иногда только так мы и можем учиться, например, когда мало данных нужного домена
      - Это плохо тогда, когда мы вносим это "случайно", например, как с неправильным усреднением, то есть это своебразный баг


## До обучения
- Выберите метрику(ки) качества и расскажите почему она(они)
    - Обычно есть основная метрика, по которой принимаем решения какие веса брать и дополнительные, которые нам помогут делать выводы, например, о том все ли хорошо с нашими данными, хорошо ли модель справляется с дисбалансом классов и тд
- Эту домашку можно сделать и на `CPU`, но на `GPU` будет сильно быстрее
    - Во всех остальных домашках мы будем учить модели на `GPU`
    - Рано или поздно вам придется посмотреть этот [туториал](https://www.youtube.com/watch?v=pgk1zGv5lU4)
    - Вы можете обучаться на `colab`, это бесплатно

## До эпохи
- Сделайте списки/словари/другое, чтобы сохранять нужные данные для расчета метрик(и) по всей эпохе для трейна и валидации

## Во время эпохи
- Используйте [`tqdm`](https://github.com/tqdm/tqdm) как прогресс бар, чтобы понимать как проходит ваше обучение
- Логируйте лосс
- Логируйте метрику(ки) по батчу
- Сохраняйте то, что вам нужно, чтобы посчитать метрик(и) на всю эпоху для трейна и валидации

## После эпохи
- Посчитайте метрик(и) на всю эпоху для трейна и валидации

## После обучения
- Провалидируйтесь на тестовом наборе и посмотрите метрики
- Постройте [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
- Постройте графики:
    - [Confusion Matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)
    - [Опционально] Распределение вероятностей мажоритарного класса (то есть для какого-то примера мы выбираем такой класс и вероятность этого выбора такая-то) на трейне/тесте/валидации
        - Если класс был выбран верно и если была ошибка
- Подумайте что еще вам будет полезно для того, чтобы ответить на такие вопросы: 
    - Что в моделе можно улучшить?
    - Все ли хорошо с моими данными?
    - Все ли хорошо с валидацией?
    - Не переобучился ли я?
    - Достаточно ли я посмотрел на данные?
    - Нужно ли мне улучшить предобработку данных?
    - Нужно ли поменять токенизацию или эмбеддинги?
    - Нет ли у меня багов в реализации?
    - Какие типичные ошибки у моей модели?
    - Как я могу их исправить?

# Я выбрал метрику <МЕТРИКА>

> Это моя метрика. Таких метрик много, но эта моя. Моя метрика — мой лучший друг. Это — моя жизнь. Я должен научиться владеть метрикой так же, как владею своей жизнью. Без меня моя метрика бесполезна. Без моей метрики бесполезен я. Я должен метко обучать модель на оптимизацию моей метрики. Я должен обучать точнее, чем конкурент, который пытается меня обойти. Я должен обучить модель лучше до того, как он обучит свою. И я это сделаю. Клянусь перед тим лидом. Я и моя метрика — мы защитники моей галеры. Мы не боимся конкурентов. Мы спасители ROI нашего отдела. Пусть будет так. Пока не останется больше конкурентов и не наступит эпоха AGI. Трансформер.

Почему я выбрал эту метрику:  
<РАССКАЗ_ПРО_МЕТРИКУ>

In [43]:
embeddings_matrix

array([[ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [-3.260e-02, -1.499e-01,  2.320e-02, ..., -2.370e-02, -2.610e-02,
        -1.832e-01],
       ...,
       [ 7.530e-02, -3.280e-02,  1.960e-02, ...,  1.000e-04, -1.710e-02,
         3.800e-03],
       [ 2.260e-02,  5.610e-02,  3.730e-02, ..., -6.020e-02,  1.041e-01,
         3.920e-01],
       [ 1.692e-01,  6.980e-02, -1.120e-02, ...,  7.210e-02, -5.685e-01,
        -2.003e-01]])

In [54]:
class DeepAverageNetwork(nn.Module):
    def __init__(self):
        super(DeepAverageNetwork, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embeddings_matrix))
        self.embeddings_dim = embeddings_matrix.shape[-1]
        self.fc1 = nn.Linear(self.embeddings_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 3)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.batchnorm1 = nn.BatchNorm1d(self.embeddings_dim)

        # self.sequential = nn.Sequential(
        #     nn.

            

    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        
        x1 = self.fc1(x)
        x1 = self.batchnorm1(x1)
        x1 = self.relu(x1)
        
   
        x2 = self.fc2(x1)
        x2 = self.batchnorm1(x2)
        x2 = self.relu(x2)
     
        x3 = self.fc3(x2)
        x3 = self.batchnorm1(x3)
        x3 = self.relu(x3)

        x4 = self.fc4(x3+x2)
        x4 = self.batchnorm1(x4)
        x4 = self.relu(x4)
        x4 = self.dropout(x4)

        
        x5 = self.fc5(x4+x1)
        x5 = self.sigmoid(x5)
       
        return x


In [55]:
model = DeepAverageNetwork()

In [56]:
model

DeepAverageNetwork(
  (embeddings): Embedding(101, 300)
  (fc1): Linear(in_features=300, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (batchnorm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

## Задайте функцию потерь и оптимизатор

In [52]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5)

## Сделайте цикл обучения

In [53]:
num_epochs = 2  # Задайте количество эпох

model.train()

for epoch in range(num_epochs):
    
    # Define the following variables to keep track of the running losses and accuracies
    running_loss = 0.0
    running_f1 = 0.0
    count = 0
    
    for i, (X, y) in enumerate(train_loader):
        
        print(i)
        if i>2:
            break
        
        # use gpu if necessary
#         if gpu_available:
#             X = X.cuda()
#             y = y.cuda()
        
        # clear the gradient buffer
        optimizer.zero_grad()
        
        # Do forward propagation to get the model's belief
        logits = model(X)
        
        # Compute the loss
        loss = loss_fn(logits, y.long())
        
        # Run a backward propagation to get the gradient
        loss.backward()
        
        # Update the model's parameter
        optimizer.step()
        
        # Get the model's prediction 
        pred = torch.argmax(logits,dim=1)
        
        # Update the running statistics
        running_f1 += f1_score(y, pred, average='weighted')
        running_loss += loss.item()
        count += X.size(0)
        
    # print the running statistics after training for 100 epochs
#     if (epoch + 1) % 100 == 0:
#         print('Epoch [{} / {}] Average Training Accuracy: {:4f}'.format(epoch + 1, \
#                                                                         num_epochs, running_acc / count))
#         print('Epoch [{} / {}] Average Training loss: {:4f}'.format(epoch + 1, \
#                                                                     num_epochs, running_loss / len(trainloader)))
    
    # train
    ...

    # validation
    ...
    
# test
...

0


RuntimeError: running_mean should contain 128 elements not 300

In [ ]:
NUM_EPOCHS = 2  # Задайте количество эпох

...

for n_epoch in range(NUM_EPOCHS):
    
    ...
    
    # train
    ...

    # validation
    ...
    
# test
...

Ellipsis

# Выводы
Напишите небольшой отчет о проделанной работе. Что удалось, в чем не уверены, что делать дальше.